# Deciding which city to move - Coursera Capstone project

This repository is used to complete the coursera ibm datascience course.

## Introducction

In this notebook I'm going to compare which city is similar to munich in the intentions to decide which city will be the appropiate to move based on similiarity of venues, things to do and its population.

The comparasion will be made between major cities of several countries. Some of them will be listed next:

- Munich <- Reference point
- Berlin
- Hamburg
- Düsseldorf
- Vienna
- Prague
- Madrid
- Barcelona
- Montreal
- Toronto
- etc.

## Data to use

For this project we will use the [foursquare api](https://foursquare.com/) to investigate which venues are each city center. 

We are going to compare the population of each city using this dataset from simple maps [World cities population](https://simplemaps.com/data/world-cities).

We need to also compare the cost of living in each city, for this we are going to use the [dataset](https://www.numbeo.com/cost-of-living/rankings.jsp) where we will use the the cost of living index to compare each city.


